# CoppeliaSim: PART 2
* Run the scene workshop1.ttt in CoppeliaSim

In [1]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Check if python is connecting to Coppelia

In [2]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    name = 'Omnirob'
    err_code, cuboid = sim.simxGetObjectHandle(clientID, name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        
    print('Omnirob is at [x,y,z]=', position)
    
    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')


Connected
Simulation time in milliseconds:  0
Omnirob is at [x,y,z]= [0.0, 0.0, 0.0]
Disconnected


# Robot

In [11]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)
    
    

In [4]:
# Joint names in coppelia for KUKA omnidirectional robot
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']


r = robot('Omnirob', motor_names)  # Create an instance of our robot

# Move the robot in a specific direction
vx = 0
vy = 0

r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])

r.close_connection()  # End

Robot connected
Connection closed


# The Code I'm Using

In [ ]:


# Time interval
time_steps = linspace(0, 70, 700)


        
    disk.set_position([tx, ty, 1])
    robot_position = r.get_position()
    u = 15*((tx, ty, 0) - robot_position) + [vx, vy, 0]
    vx, vy, vz = u
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    time.sleep(0.1)

r.send_motor_velocities([0, 0, 0, 0])
r.close_connection()  # End

# Trajectory follower

A {straight-line trajectory} is a function that describes a segment of a straight line in the Euclidean space that starts at a point $\mathbf{p}_0$ at $t=0$ and ends at a point $\mathbf{p}_f$ at time $t=t_f$. The equation that describes the trajectory is:
	\begin{equation}
	\mathbf{\gamma}(t) = \mathbf{a}_1 t + \mathbf{a}_0,
	\label{eq:line}
	\end{equation}
	where $\mathbf{a}_1=\frac{\mathbf{p}_f - \mathbf{p}_0}{t_f}$ and $\mathbf{a}_0=\mathbf{p}_0$, for $t\in[0,t_f]$. 	

In [127]:

def point_to_point_traj(x1, x2, v1, v2, delta_t):
    t = np.linspace(0, delta_t, 100)  
          # the following is finding each coefficient for the spline
    a0 = x1
    a1 = v1
    a2 = (3*x2 - 3*x1 - 2*v1*delta_t - v2 * delta_t) / (delta_t**2)
    a3 = (2*x1 + (v1 + v2) * delta_t  - 2 * x2) / (delta_t**3)
    return a0, a1, a2, a3
    
def piecewise2D (X,Y, Vx, Vy, T):
    a0x, a1x, a2x, a3x = [0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0]
    a0y, a1y, a2y, a3y = [0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0,0]

    for i in range(len(P)-1):          
        a0xi, a1xi, a2xi, a3xi = point_to_point_traj(X[i], X[i+1], Vx[i], Vx[i+1], T[i+1] - T[i])
        a0yi, a1yi, a2yi, a3yi = point_to_point_traj(Y[i], Y[i+1], Vy[i], Vy[i+1], T[i+1] - T[i])

        a0x[i] = a0xi
        a1x[i] = a1xi
        a2x[i] = a2xi
        a3x[i] = a3xi
        
        a0y[i] = a0yi
        a1y[i] = a1yi
        a2y[i] = a2yi
        a3y[i] = a3yi
        
            
    return a0x, a1x, a2x, a3x, a0y, a1y, a2y, a3y

# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot('Omnirob', motor_names)  # Create an instance of our robot

# Initial locations
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

# Waypoints
p1 = [7.625,8.55]
p2 = [0.2,7.75]
p3 = [-3.9,3.4]
p4 = [1.925,2.225]
p5 = [5,-2.5]
p6 = [1, -5]
p7 = [6.125, -6.15]
p8 = [2.12, -6.1]
p9 = [-2.023, -3.223] # an extra one so we don't hit anything
p10 = [-6.65, -8.5]

# Velocities
v1 = [-0.2,-0.2]
v2 = [-1,-0.5]
v3 = [0,-1.5]
v4 = [1.7, -0.3]
v5 = [-2.8,-0.6]
v6 = [0.5, -1.5]
v7 = [-0.2, -0.2]
v8 = [-0.2, 1]
v9 = [-1, -1]
v10 = [-1,0.1]

# Time
t1 = 0
t2 = 10
t3 = 20
t4 = 30
t5 = 40
t6 = 45
t7 = 50
t8 = 55
t9 = 60
t10 = 70


# Convert the initial conditions to a vector form
P = np.vstack((p1, p2, p3, p4, p5, p6, p7, p8, p9, p10))
V = np.vstack((v1, v2, v3, v4, v5, v6, v7, v8, v9, v10))
T = [t1, t2, t3, t4, t5, t6, t7, t8, t9, t10]

X, Y = P[:,0], P[:,1]
Vx, Vy = V[:,0], V[:,1]

a0x, a1x, a2x, a3x, a0y, a1y, a2y, a3y = piecewise2D(X,Y, Vx, Vy, T)


# Virtual disk
disk = robot('Disc', client_id=r.client_id)

time_stepping = linspace(0, 5, 50)
for a in time_stepping:
    if r.client_id == -1: # Check connection
        print('Robot is not connected')
        break
    elif a < 2.3000:
        r.send_motor_velocities([1.00, 1.00, 1.00, 1.00])
    else:
        break
    time.sleep(0.1)

# Time interval
time_steps = linspace(0, 80, 800)

for t in time_steps:
    if r.client_id == -1:  # Check connection
        print('Robot is not connected')
        break
    elif t < 10:
        tx, ty = a0x[0] + a1x[0] * t + a2x[0] * t**2 + a3x[0] * t**3, a0y[0] + a1y[0] * t + a2y[0] * t**2 + a3y[0] * t**3
        vx, vy = a1x[0] + 2*a2x[0] * t + 3 * a3x[0] * t**2, a1y[0] + 2*a2y[0] * t + 3 * a3y[0] * t**2
    elif t < 20:
        tx, ty = a0x[1] + a1x[1] * (t-10) + a2x[1] * (t-10)**2 + a3x[1] * (t-10)**3, a0y[1] + a1y[1] * (t-10) + a2y[1] * (t-10)**2 + a3y[1] * (t-10)**3
        vx, vy = a1x[1] + 2*a2x[1] * (t-10) + 3 * a3x[1] * (t-10)**2, a1y[1] + 2*a2y[1] * (t-10) + 3 * a3y[1] * (t-10)**2
    elif t < 30:
        tx, ty = a0x[2] + a1x[2] * (t-20) + a2x[2] * (t-20)**2 + a3x[2] * (t-20)**3, a0y[2] + a1y[2] * (t-20) + a2y[2] * (t-20)**2 + a3y[2] * (t-20)**3
        vx, vy = a1x[2] + 2*a2x[2] * (t-20) + 3 * a3x[2] * (t-20)**2, a1y[2] + 2*a2y[2] * (t-20) + 3 * a3y[2] * (t-20)**2
    elif t < 40:
        tx, ty = a0x[3] + a1x[3] * (t-30) + a2x[3] * (t-30)**2 + a3x[3] * (t-30)**3, a0y[3] + a1y[3] * (t-30) + a2y[3] * (t-30)**2 + a3y[3] * (t-30)**3
        vx, vy = a1x[3] + 2*a2x[3] * (t-30) + 3 * a3x[3] * (t-30)**2, a1y[3] + 2*a2y[3] * (t-30) + 3 * a3y[3] * (t-30)**2
    elif t < 45:
        tx, ty = a0x[4] + a1x[4] * (t-40) + a2x[4] * (t-40)**2 + a3x[4] * (t-40)**3, a0y[4] + a1y[4] * (t-40) + a2y[4] * (t-40)**2 + a3y[4] * (t-40)**3
        vx, vy = a1x[4] + 2*a2x[4] * (t-40) + 3 * a3x[4] * (t-40)**2, a1y[4] + 2*a2y[4] * (t-40) + 3 * a3y[4] * (t-40)**2
    elif t < 50:
        tx, ty = a0x[5] + a1x[5] * (t-45) + a2x[5] * (t-45)**2 + a3x[5] * (t-45)**3, a0y[5] + a1y[5] * (t-45) + a2y[5] * (t-45)**2 + a3y[5] * (t-45)**3
        vx, vy = a1x[5] + 2*a2x[5] * (t-45) + 3 * a3x[5] * (t-45)**2, a1y[5] + 2*a2y[5] * (t-45) + 3 * a3y[5] * (t-45)**2
    elif t < 55:
        tx, ty = a0x[6] + a1x[6] * (t-50) + a2x[6] * (t-50)**2 + a3x[6] * (t-50)**3, a0y[6] + a1y[6] * (t-50) + a2y[6] * (t-50)**2 + a3y[6] * (t-50)**3
        vx, vy = a1x[6] + 2*a2x[6] * (t-50) + 3 * a3x[6] * (t-50)**2, a1y[6] + 2*a2y[6] * (t-50) + 3 * a3y[6] * (t-50)**2
    elif t < 60:
        tx, ty = a0x[7] + a1x[7] * (t-55) + a2x[7] * (t-55)**2 + a3x[7] * (t-55)**3, a0y[7] + a1y[7] * (t-55) + a2y[7] * (t-55)**2 + a3y[7] * (t-55)**3
        vx, vy = a1x[7] + 2*a2x[7] * (t-55) + 3 * a3x[7] * (t-55)**2, a1y[7] + 2*a2y[7] * (t-55) + 3 * a3y[7] * (t-55)**2
    elif t < 70:
        tx, ty = a0x[8] + a1x[8] * (t-60) + a2x[8] * (t-60)**2 + a3x[8] * (t-60)**3, a0y[8] + a1y[8] * (t-60) + a2y[8] * (t-60)**2 + a3y[8] * (t-60)**3
        vx, vy = a1x[8] + 2*a2x[8] * (t-60) + 3 * a3x[8] * (t-60)**2, a1y[8] + 2*a2y[8] * (t-60) + 3 * a3y[8] * (t-60)**2
    else:
        break;
        
    disk.set_position([tx, ty, 1])
    robot_position = r.get_position()
    u = 5*((tx, ty, 0) - robot_position) + [vx, vy, 0]
    vx, vy, vz = u
    r.send_motor_velocities([-vx + vy, vx + vy, vx - vy, -vx - vy])
    time.sleep(0.1)


r.close_connection() 

Robot connected
Robot position: (7.62, 8.55) 
Robot connected
Connection closed


In [8]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot('Omnirob', motor_names)  # Create an instance of our robot

# Connect to the sphere that will be a reference
ref_point = robot('Sphere11', client_id=r.client_id)

# Initial locations
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))
desired_position = r.get_object_position('Sphere1')
print('Desired position: (%.2f, %.2f) '%(desired_position[0], desired_position[1]))
relative_position = r.get_object_relative_position('Sphere1')
print('Relative position: (%.2f, %.2f) '%(relative_position[0], relative_position[1]))


tf = 10.
tstep = 0.1
a1 = (desired_position - robot_position) / tf
a0 = robot_position



# Time interval
time_steps = linspace(0, tf, int(tf/tstep))

for t in time_steps:
    # Compute the trajectory
    traj_point = a1*t + a0
    vel_traj = a1
    # Show the trajectory point
    ref_point.set_position(traj_point)
    
    # Location sesing
    robot_position = r.get_position()
    
    # drive the robot using the trajectory tracker
    u = 0.5*(traj_point - robot_position) + vel_traj
    
    
    vx, vy, vz = u
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
    time.sleep(tstep)


    
r.send_motor_velocities([0, 0, 0, 0])

r.close_connection()  # End

Robot connected
Robot connected
Robot position: (7.62, 8.55) 
Desired position: (0.40, 8.05) 
Relative position: (-7.22, -0.50) 
Connection closed
